# Les bases de pandas pour faire interagir Excel avec Python

## Selectionner les données

In [8]:
import pandas as pd

### Sélection simple

Pour sélectionner une colonne spécifique dans un DataFrame, vous pouvez utiliser la syntaxe suivante :

```python
nom_data_frame['nom_colonne']
```

Vous pouvez sélectionner plusieurs colonnes en spécifiant une liste de noms de colonnes :

```python
nom_data_frame[['nom_colonne1', 'nom_colonne2', ...]]
```

Vous pouvez sélectionner des lignes spécifiques en utilisant leur index avec la méthode loc[] :

```python
nom_data_frame.loc[index]
```

### _Exercice 1_

Importer le fichier référentiel.xlsx dans un dataframe et sélectionnez : 

* Les trois premières lignes du DataFrame.

In [18]:
df_reference = pd.read_excel("referentiel.xlsx", header = 0)
three_lines = df_reference.loc[:2]

In [19]:
three_lines.head()

,Unnamed: 0,Pays_Cod,Pays,Région,Sous-Région,Code Sous-Région,Unnamed: 6,Product_Ref,Produit,Catégorie de produit,Couleur,Créé le,Prix d'achat,Prix vente HT
0,NaN,BGR,Bulgarie,Europe,Europe de l'Est,EUE,NaN,P00147,Robe,Haut-Et-Bas,marron,2018-01-01,7.20,9
1,NaN,CZE,République Tchèque,Europe,Europe de l'Est,EUE,NaN,P00249,Chemise,Haut,blanc,2018-06-01,11.34,14
2,NaN,HUN,Hongrie,Europe,Europe de l'Est,EUE,NaN,P00565,Robe,Haut-Et-Bas,orange,2017-05-01,13.05,15


* les trois dernières colonnes

In [21]:
last_three_columns = df_reference[['Créé le',"Prix d'achat","Prix vente HT"]]
last_three_columns.head()

,Créé le,Prix d'achat,Prix vente HT
0,2018-01-01,7.20,9
1,2018-06-01,11.34,14
2,2017-05-01,13.05,15
3,2018-12-01,5.32,7
4,2018-04-01,14.25,15


## Sélection conditionnelle

Vous pouvez sélectionner des lignes qui répondent à certaines conditions à l'aide de la méthode loc[] avec une expression conditionnelle :

```python
nom_data_frame.loc[condition]
```

Vous pouvez combiner plusieurs conditions à l'aide d'opérateurs logiques (& pour ET, | pour OU) :

```python
nom_data_frame.loc[(condition1) & (condition2)]
```

### _Exercice 2_

Sélectionnez dans le référentiel : 

* Les produits dont le prix de vente est supérieur à 15 euros

In [28]:
fifteen_plus = df_reference.loc[df_reference["Prix vente HT"] >= 10]

In [29]:
fifteen_plus.head()

,Unnamed: 0,Pays_Cod,Pays,Région,Sous-Région,Code Sous-Région,Unnamed: 6,Product_Ref,Produit,Catégorie de produit,Couleur,Créé le,Prix d'achat,Prix vente HT
1,NaN,CZE,République Tchèque,Europe,Europe de l'Est,EUE,NaN,P00249,Chemise,Haut,blanc,2018-06-01,11.34,14
2,NaN,HUN,Hongrie,Europe,Europe de l'Est,EUE,NaN,P00565,Robe,Haut-Et-Bas,orange,2017-05-01,13.05,15
4,NaN,POL,Pologne,Europe,Europe de l'Est,EUE,NaN,P00626,Robe,Haut-Et-Bas,blanc,2018-04-01,14.25,15
8,NaN,UKR,Ukraine,Europe,Europe de l'Est,EUE,NaN,P00924,Chaussette,Bas,bleu,2018-05-01,13.65,15
9,NaN,ARM,Arménie,Europe,Europe de l'Est,EUE,NaN,P01048,Culotte,Bas,orange,2018-09-01,12.88,14


* Les produits de couleur marron

In [32]:
brown = df_reference.loc[df_reference["Couleur"] == "marron"]

In [33]:
brown.head()

,Unnamed: 0,Pays_Cod,Pays,Région,Sous-Région,Code Sous-Région,Unnamed: 6,Product_Ref,Produit,Catégorie de produit,Couleur,Créé le,Prix d'achat,Prix vente HT
0,NaN,BGR,Bulgarie,Europe,Europe de l'Est,EUE,NaN,P00147,Robe,Haut-Et-Bas,marron,2018-01-01,7.20,9
12,NaN,EST,Estonie,Europe,Europe du Nord,EUN,NaN,P01596,Collant,Bas,marron,2018-08-01,6.37,7
17,NaN,LTU,Lituanie,Europe,Europe du Nord,EUN,NaN,P01933,Soutien gorge,Haut,marron,2017-08-01,6.48,9
27,NaN,SVN,Slovénie,Europe,Europe du Sud,EUS,NaN,P03438,Robe,Haut-Et-Bas,marron,2017-10-01,8.37,9
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P06356,Pull,Haut,marron,2018-03-01,10.78,14


## Selection conditionnelle avancée

Imaginons que nous voulons créer des catégories de gamme en fonction des valeurs de prix des produits. Les produits qui se situe dans le tiers supérieur en termes de prix sont qualifiés de "haut de gamme", ceux du tiers intermédiaires comme "milieu de gamme", ceux du tiers inférieur comme "bas de gamme". 

#### Étape 1 : Création des trois catégories

Nous allons diviser nos produits en trois catégories de gamme : haut de gamme, milieu de gamme et bas de gamme. Pour cela, nous devons calculer les quantiles des prix d'achat pour déterminer les limites entre ces catégories.

```python
quantiles = donnees['Prix_achat'].quantile([0.33, 0.66])
```

#### Étape 2 : Attribution de gamme aux produits

Nous allons définir une fonction qui attribue une catégorie de gamme à chaque produit en fonction de son prix d'achat.

```python
def assigner_gamme(row):
    if row['Prix_achat'] >= quantiles.iloc[0.66]:
        return 'haut de gamme'
    elif row['Prix_achat'] >= quantiles.iloc[0.33]:
        return 'milieu de gamme'
    else:
        return 'bas de gamme'
```

#### Interlude : la méthode apply

La méthode apply est une fonction de Pandas qui permet d'appliquer une fonction personnalisée à tout élément d'un DataFrame. Pour l'utiliser : 

```python
nom_dataframe.apply(nom_de_la_fonction, axis=axis)
```

*   `nom_dataframe` : C'est le DataFrame sur lequel vous souhaitez appliquer la fonction.
*   `nom_de_la_fonction` : C'est le nom de la fonction que vous souhaitez appliquer à chaque élément, ligne ou colonne du DataFrame.
*   `axis` : C'est un paramètre optionnel qui spécifie l'axe sur lequel appliquer la fonction. Par défaut, `axis=0` applique la fonction sur chaque colonne, tandis que `axis=1` l'applique sur chaque ligne.

#### Étape 3 : Application de la fonction à chaque ligne du DataFrame

Appliquons maintenant notre fonction à chaque ligne du DataFrame pour créer une nouvelle colonne "gamme".

```python
donnees['gamme'] = donnees.apply(assigner_gamme, axis=1)
```

#### Résultat final

Voici le DataFrame final avec la nouvelle colonne "gamme" ajoutée :

```python
print(donnees)
```

## Agréger toutes les données

Dans le monde réel, les données vous seront toujours envoyées dans plusieurs fichiers, et vous aurez très certainement besoin d'agréger ces fichiers pour avoir une vision d'ensemble et faire des analyses. 

#### Etape 1 : La méthode `concat`

Si vous voulez simplement coller plusieurs fichiers entre eux, la méthode `concat` est ici la plus utile. 

Voici comment utiliser la méthode `concat` avec des données provenant de deux fichiers Excel que voulez coller parce qu'ils contiennent les mêmes variables sur un phénomène, seulement à des dates diffférentes :

```python
import pandas as pd

# Charger les données à partir du fichier Excel
df1 = pd.read_excel('vente_2021.xlsx')
df2 = pd.read_excel('ventes_2022.xlsx')

# Concaténer les DataFrames le long de l'axe des lignes (par défaut)
resultat = pd.concat([df1, df2])

print(resultat)
```

Maintenant, voici comment utiliser la méthode `concat` avec des fichiers excel sur le même phénomène mais avec des variables différentes

```python
# Lecture des données des fichiers Excel pour les années 2021 et 2022
df_covid_2021_read = pd.read_excel('covid_2021.xlsx')
df_covid_2022_read = pd.read_excel('covid_2022.xlsx')

# Concaténation des données sur l'axe des colonnes
df_concat = pd.concat([df_covid_2021_read, df_covid_2022_read], axis=1)
```



#### _Exercice 3_


Considérez les deux DataFrames suivants issus de tableurs excel :

DataFrame 1 :
```python
import pandas as pd

df1 = pd.DataFrame({'Pays': ['France', 'Allemagne', 'Italie'],
                    'Capitale': ['Paris', 'Berlin', 'Rome']})
```

DataFrame 2 :
```python
df2 = pd.DataFrame({'Population': [67000000, 83000000, 60000000],
                    'Superficie': [551695, 357022, 301338]})
```


* Concaténez les DataFrames `df1` et `df2` le long de l'axe des lignes.

```python
resultat_task1 = pd.concat([df1, df2])
print(resultat_task1)
```

* Concaténez les DataFrames `df1` et `df2` le long de l'axe des colonnes.

```python
resultat_task2 = pd.concat([df1, df2], axis=1)
print(resultat_task2)
```

* Concaténez les DataFrames `df1` et `df2` le long de l'axe des colonnes en réindexant les colonnes.

```python
resultat_task3 = pd.concat([df1, df2], axis=1, ignore_index=True)
resultat_task3.columns = ['Pays', 'Capitale', 'Population', 'Superficie']
print(resultat_task3)
```


Créeons un nouveau DataFrame `df3` avec les données suivantes :

```python
df3 = pd.DataFrame({'Langue': ['Français', 'Allemand', 'Italien'],
                    'Monnaie': ['Euro', 'Euro', 'Euro']})
```
* Concaténez `df1` et `df3` le long de l'axe des colonnes, en conservant les index d'origine.

```python
resultat_task4 = pd.concat([df1, df3], axis=1)
print(resultat_task4)
```

#### Etape 2 : Les méthodes `join`

La méthode `join` permet de joindre deux DataFrames sur leurs index. 

Voici comment utiliser la méthode `join` avec des données provenant de fichiers Excel :

```python
# Charger les données à partir du fichier Excel
df1 = pd.read_excel('fichier_excel.xlsx', sheet_name='Feuille1')
df2 = pd.read_excel('fichier_excel.xlsx', sheet_name='Feuille2')

# Joindre les DataFrames sur une colonne spécifiée
resultat = df1.join(df2, how='type_de_jointure')

print(resultat)
```

Il existe différents types de jointures, utiles dans des cas différents : 

- **Jointure interne :** Elle renvoie uniquement les lignes où il existe une correspondance dans les deux DataFrames. Utile pour combiner des données sur des clés communes.
- **Jointure externe (full outer join) :** Elle renvoie toutes les lignes des deux DataFrames, avec des valeurs NaN là où il n'y a pas de correspondance. Utile pour combiner toutes les données et ne pas perdre d'informations.
- **Jointure gauche (left join) :** Elle renvoie toutes les lignes du DataFrame de gauche (premier DataFrame spécifié dans la méthode `merge` ou `join`), avec des valeurs NaN là où il n'y a pas de correspondance dans le DataFrame de droite. Utile pour ajouter des informations à un DataFrame principal.
- **Jointure droite (right join) :** Elle renvoie toutes les lignes du DataFrame de droite, avec des valeurs NaN là où il n'y a pas de correspondance dans le DataFrame de gauche. Utile pour ajouter des informations à un DataFrame secondaire.
- **Combinaison de plusieurs jointures :** Parfois, il est nécessaire de combiner plusieurs jointures pour obtenir les données souhaitées, surtout lorsque les données sont distribuées sur plusieurs DataFrames.

#### _Exercice 4_

Considérons les deux DataFrames suivants :

DataFrame 1 :
```python
import pandas as pd

df1 = pd.DataFrame({'ID': [1, 2, 3],
                    'Nom': ['Alice', 'Bob', 'Charlie']},
                   index=['A', 'B', 'C'])
```

DataFrame 2 :
```python
df2 = pd.DataFrame({'Employee_ID': [1, 3, 4],
                    'Département': ['RH', 'Finance', 'Marketing']},
                   index=['A', 'B', 'D'])
```


* Effectuez une jointure entre `df1` et `df2` sur leurs index.


```python
resultat_exercice1 = df1.join(df2)
print(resultat_exercice1)
```

* Utilisez la méthode `.join` pour joindre les DataFrames `df1` et `df2` sur leurs index en spécifiant une jointure externe.

```python
resultat_exercice3 = df1.join(df2, how='outer')
print(resultat_exercice3)
```


#### Etape 2 : La méthode `merge`


La méthode `merge` différe de `join` dans la mesure où elle permet de combiner des données provenant de différentes sources en utilisant une ou plusieurs colonnes comme clés de jointure.

Syntaxe de base :

```python
resultat_jointure = pd.merge(df1, df2, on='colonne_commun', how='type_de_jointure')
```

- `df1` et `df2` : Les DataFrames à fusionner.
- `on` : Le nom de la colonne commune sur laquelle effectuer la jointure.
- `how` : Le type de jointure à effectuer : 'inner' (par défaut), 'outer', 'left' ou 'right'.

Exemple d'utilisation :

Supposons que nous ayons deux DataFrames : `employes` et `departements`.

```python
import pandas as pd

employes = pd.DataFrame({'ID': [1, 2, 3],
                         'Nom': ['Alice', 'Bob', 'Charlie']})

departements = pd.DataFrame({'Employee_ID': [1, 3, 4],
                             'Département': ['RH', 'Finance', 'Marketing']})
```

Nous pouvons utiliser la méthode `merge` pour fusionner ces DataFrames sur la colonne 'ID' de `employes` et la colonne 'Employee_ID' de `departements`.

```python
resultat_jointure = pd.merge(employes, departements, left_on='ID', right_on='Employee_ID', how='inner')
print(resultat_jointure)
```


## Interroger les données (dans tous les sens)

In [88]: df.groupby(["continent"]).mean()

A popular way to get statistics per group in Excel is to use pivot tables.

 In [92]: pivot = pd.pivot_table(sales,
                                    index="Fruit", columns="Region",
                                    values="Revenue", aggfunc="sum",
                                    margins=True, margins_name="Total")
pivot
    Out[92]: Region   North  South  Total
             Fruit
             Apples     NaN  10.55  10.55
             Bananas   31.3   5.90  37.20
             Oranges   25.4  22.00  47.40
             Total     56.7  38.45  95.15

Working with aggregated statistics helps you understand your data, but nobody likes to read a page full of numbers.

...

# Mise en pratique

Quand on ouvre des fichiers Excel pour la première fois, il est souvent difficile de leur donner du sens. Les données que l'on reçoit sont rarement calibrées pour notre manière personnelle de comprendre les choses. 

Voyez pandas comme un outil ultra puissant qui va permettre de customiser des données pour vous. 

Mettons que votre boss vient de vous envoyer ces deux fichier Excel 👇

**Fichier 1**

<img src="media/ventes.png" width="500"/>

**Fichier 2**

<img src="media/referentiel.png" width="500"/>

Vous voulez comprendre **tout ce qu'il est possible de comprendre** sur l'activité de l'entreprise à partir de ces deux fichiers (en un temps record).

In [3]:
import pandas as pd